# Get Statistics on Existing Data

Before we proceed with conversion of the data to a format that can be used by the NoSketch Engine let's first get some statistics on the existing data. This will help us understand the data better and also help us in making decisions on how to proceed with the data conversion.

In [5]:
from pathlib import Path
import zipfile
# TODO Write a function that takes a zipfile and provides statistics on its contents
# The function should return a dictionary with the following keys
# - number_of_first_level_folders (this would indicate how many individual issues are in the zip) 
# - number_of_files_across_all_folders
# - total_size
# - largest_file - value would be tuple with file name and size
# - smallest_file - value would be tuple with file name and size
# - files_with_no_extension
# - common_extensions - value would be a list of tuples with extension and count
# TODO could add more keys if you think they would be useful

# it would be nice to also have the following optional keys, provided as arguments to the function
# suffix_with_extension_count
# for example _ocr_xml_count - would be a count of files ending with _ocr.xml - 
# note how key is constructed from suffix but uses _ instead of . where applicable

def get_stats(src_zip: str|Path, suffixes=("_ocr.xml","_alto.xml")) -> dict:
    stats_dict = {}
    # TODO
    return stats_dict


In [6]:
# TODO
# when get_stats function is written
# next step would be to write a function that takes a folder and provides statistics on all zip files in the folder and its subfolders
# the function should return a dictionary with the following keys
# - number_of_zip_files
# - total_size
# - largest_zip_file - value would be tuple with file name and size
# - smallest_zip_file - value would be tuple with file name and size
# - zip_file_stats - value would be a dictionary where keys are zip file names and values are the output of get_stats function (another dictionary)
# thus zip_file_stats key value would be a dictionary of dictionaries

def get_stats_on_zips(src_folder: str|Path) -> dict:
    zip_dict = {}
    # TODO
    return zip_dict

## Statistics on Articles subfolder

Our institutution has a large number of plaintext articles already available and stored in our local system. These articles are stored in the `articles` subfolder. Let's first get some statistics on these articles.


In [ ]:
# TODO
# Generate statistics on the zip files in the articles subfolder of the zips folder
# example of file names inside the individual zip files
# cina1991n110-111_029_plaintext_s45.txt
# cina1991n109_016_plaintext_s25.txt
# the letters before the number indicate the publication name or its shorthand
# then there is a four digit number indicating the year
# then there is a letter n followed by a number indicating the issue
# note that sometimes there are two issues together indicated by - and two numbers
# then there is an underscore followed by a number indicating the page - we are not interested in this number at this time
# thus we want to write two functions
# first will get stats on individual zip files
# second will get stats on all zip files in the articles subfolder of the zips folder

# TODO
def get_article_stats(zip_file: str|Path) -> dict:
    article_stats_dict = {}
    # TODO
    # we want the following keys:
    # number_of_articles
    # number_of_issues (that is the number of unique publication name and year combinations)
    # min_year
    # max_year
    # year_XXXX_issue_count (where XXXX is the year) for example year_1991_count
    # years_active (a list of years that have issues)
    # min_file_size
    # max_file_size
    # median_file_size
    # anything else you think would be useful
    return article_stats_dict


def get_stats_on_articles(src_folder: str|Path) -> dict:
    article_dict = {}
    # for each file in the articles subfolder of the zips folder
    # get the stats using get_article_stats function
    # use keys that are the file names
    for file in Path(src_folder).glob("*.zip"):
        article_dict[file.name] = get_article_stats(file)
    return article_dict

# convert article_dict to a pandas dataframe
# write the dataframe to a csv file